In [35]:
!pip install scapy

In [36]:
import pandas as pd
import numpy as np

# 1. טעינת הקובץ שלך
# וודא ששם הקובץ בקוד זהה לשם הקובץ ששמרת במחשב
filename = "group209186378_http_input.csv" 
messages_df = pd.read_csv(filename)

# 2. מילון תרגום: הופך שמות של אפליקציות למספרי פורטים
# זה החלק שמגשר בין דרישות ה-PDF לדרישות הקוד
app_port_map = {
    "client_browser": 55000,  # סתם פורט גבוה שמדמה דפדפן
    "web_server": 80          # הפורט הסטנדרטי של HTTP
}

# 3. יצירת עמודות הפורטים (src_port, dst_port) שהקוד בהמשך חייב לקבל
messages_df['src_port'] = messages_df['src_app'].map(app_port_map).fillna(12345).astype(int)
messages_df['dst_port'] = messages_df['dst_app'].map(app_port_map).fillna(80).astype(int)

# מילוי הודעות ריקות כדי למנוע שגיאות
messages_df['message'] = messages_df['message'].fillna('')

print("הקובץ נטען בהצלחה! הנה התוצאה עם הפורטים:")
print(messages_df[['src_app', 'src_port', 'dst_app', 'dst_port', 'message']].head())

הקובץ נטען בהצלחה! הנה התוצאה עם הפורטים:
          src_app  src_port         dst_app  dst_port  \
0  client_browser     55000      web_server        80   
1      web_server        80  client_browser     55000   
2  client_browser     55000      web_server        80   
3      web_server        80  client_browser     55000   
4  client_browser     55000      web_server        80   

                      message  
0    GET /index.html HTTP/1.1  
1             HTTP/1.1 200 OK  
2     GET /style.css HTTP/1.1  
3  HTTP/1.1 200 OK (CSS Data)  
4        POST /login HTTP/1.1  


In [37]:
import socket, struct, random, time, platform
from typing import Optional

# פונקציה לחישוב Checksum (חובה לפרוטוקול TCP/IP)
def checksum(data: bytes) -> int:
    if len(data) % 2:
        data += b'\0'
    res = sum(struct.unpack('!%dH' % (len(data)//2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return ~res & 0xFFFF

# בדיקה אם אנחנו על ווינדוס (דורש טיפול מיוחד)
IS_WINDOWS = (platform.system() == 'Windows')
try:
    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send
    HAVE_SCAPY = True
except:
    HAVE_SCAPY = False

In [38]:
def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int=socket.IPPROTO_TCP) -> bytes:
    version_ihl = (4 << 4) + 5
    total_length = 20 + payload_len
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, 0, total_length, 54321,
                             0, 64, proto, 0,
                             socket.inet_aton(src_ip), socket.inet_aton(dst_ip))
    chksum = checksum(ip_header)
    # הרכבה מחדש עם ה-Checksum התקין
    return struct.pack('!BBHHHBBH4s4s',
                        version_ihl, 0, total_length, 54321,
                        0, 64, proto, chksum,
                        socket.inet_aton(src_ip), socket.inet_aton(dst_ip))

def build_tcp_header(src_ip: str, dst_ip: str, src_port: int, dst_port: int, payload: bytes=b'', flags: int=0x02) -> bytes:
    seq = random.randint(0, 4294967295)
    tcp_header = struct.pack('!HHLLBBHHH', src_port, dst_port, seq, 0, (5 << 4), flags, 8192, 0, 0)
    
    # Pseudo header for checksum calculation
    pseudo_header = struct.pack('!4s4sBBH',
                                socket.inet_aton(src_ip), socket.inet_aton(dst_ip),
                                0, socket.IPPROTO_TCP, len(tcp_header) + len(payload))
    chksum = checksum(pseudo_header + tcp_header + payload)
    
    return struct.pack('!HHLLBBHHH', src_port, dst_port, seq, 0, (5 << 4), flags, 8192, chksum, 0)

In [39]:
class RawTcpTransport:
    def __init__(self, src_ip, dst_ip, src_port, dst_port):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
    
    def send(self, data: bytes, flags: int=0x18):
        # Windows Logic (Scapy)
        if IS_WINDOWS:
            if not HAVE_SCAPY:
                print("Error: Scapy not installed. Run: pip install scapy")
                return
            pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip)/ \
                  SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags)/ \
                  SCAPY_Raw(data)
            # שימוש במתאם Loopback
            scapy_send(pkt, verbose=False, iface="Adapter for loopback traffic capture")
        
        # Linux/Mac Logic (Raw Sockets)
        else:
            sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
            tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags)
            ip = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
            sock.sendto(ip + tcp + data, (self.dst_ip, 0))

In [41]:
# תא 5: ביצוע השליחה
print("מתחיל בשליחת הודעות מה-CSV...")

# יצירת אובייקט השליחה
transport = RawTcpTransport('127.0.0.1', '127.0.0.1', 0, 0)

for index, row in messages_df.iterrows():
    # שליפת נתונים
    msg = str(row['message'])
    # שימוש בפורטים שיצרנו בתא הראשון (וודא שהרצת אותו!)
    s_port = int(row['src_port']) 
    d_port = int(row['dst_port'])
    
    # עדכון פורטים ושליחה
    transport.src_port = s_port
    transport.dst_port = d_port
    
    print(f"Sending: {msg}")
    transport.send(msg.encode('utf-8'))
    time.sleep(1) # השהייה כדי שיהיה ברור ב-Wireshark

print("הסתיים.")

מתחיל בשליחת הודעות מה-CSV...
Sending: GET /index.html HTTP/1.1
Sending: HTTP/1.1 200 OK
Sending: GET /style.css HTTP/1.1
Sending: HTTP/1.1 200 OK (CSS Data)
Sending: POST /login HTTP/1.1
הסתיים.
